In [3]:
import pandas as pd
import numpy as np

def calculate_dmd_matrix_A(file_x, file_xprime):
    
    print(f"Wczytywanie danych z plików: {file_x}, {file_xprime}...")
    
    try:
        df_x = pd.read_csv(file_x, sep=';', decimal=',', header=None)
        df_xprime = pd.read_csv(file_xprime, sep=';', decimal=',', header=None)
        
        X = df_x.values
        Xprime = df_xprime.values
    except Exception as e:
        return f"Błąd podczas wczytywania plików: {e}"

    print(f"Wymiary X: {X.shape}, Wymiary X': {Xprime.shape}")

    # KROK 1: Dekompozycja SVD macierzy X
    U, s, Vh = np.linalg.svd(X, full_matrices=False)

    # KROK 2: Wybór rzędu r (rank truncation)
    threshold = s[0] * 1e-15
    r = np.sum(s > threshold)
    print(f"Zastosowany rząd przybliżenia (r): {r}")

    Ur = U[:, :r]
    sr = s[:r]
    Vhr = Vh[:r, :]

    # KROK 3: Obliczenie macierzy A
    A = Xprime @ Vhr.T @ np.diag(1/sr) @ Ur.T

    # KROK 4: Weryfikacja błędu
    X_approx = A @ X
    error = np.linalg.norm(Xprime - X_approx) / np.linalg.norm(Xprime)
    
    print(f"Obliczono macierz A o wymiarach: {A.shape}")
    print(f"Błąd względny dopasowania ||X' - AX|| / ||X'||: {error:.2e}")

    return A

file_x = 'War8_X.csv'
file_xprime = 'War8_Xprime.csv'

# Uruchomienie obliczeń
A_matrix = calculate_dmd_matrix_A(file_x, file_xprime)

if isinstance(A_matrix, np.ndarray):
    
    # Wyświetlenie fragmentu macierzy
    print("\nFragment macierzy A (lewy górny róg 5x5):")
    print(A_matrix[:5, :5])

Wczytywanie danych z plików: War8_X.csv, War8_Xprime.csv...
Wymiary X: (23, 36), Wymiary X': (23, 36)
Zastosowany rząd przybliżenia (r): 9
Obliczono macierz A o wymiarach: (23, 23)
Błąd względny dopasowania ||X' - AX|| / ||X'||: 8.29e-12

Fragment macierzy A (lewy górny róg 5x5):
[[ -90936.28117577   78264.8469396   -81656.22704274  170682.35734865
  -206828.12232566]
 [-105207.18237036   90519.77860623  -94457.61035122  197449.31146084
  -239268.95924302]
 [-105166.56367299   90517.15571859  -94432.29519274  197398.79928868
  -239181.76402281]
 [ -99589.74554409   85692.68091355  -89417.39896339  186922.25476855
  -226491.59540379]
 [ -77189.98685585   66414.14651935  -69305.82708361  144869.85518453
  -175538.34633235]]
